# Build a basic Agent
In this notebook, we'll:
1. Configure your API key to use the Gemini model
2. Build your first simple agent
3. Run the agent and watch it use a tool (like Google Search) to answer a question

## Part 1: Add Gemini API key to the env

This notebook uses the Gemini API, which requires authentication.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to env and load as shown in the script 


In [1]:
import platform
import sys

print(f"Python version: {sys.version.split()[0]} on {platform.platform()}")
%pip install -q google-adk google-genai python-dotenv

Python version: 3.13.0 on macOS-26.0.1-arm64-arm-64bit-Mach-O
Note: you may need to restart the kernel to use updated packages.


## Part 1: Add Gemini API key to the env

This notebook uses the Gemini API, which requires authentication.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to env and load as shown in the script 


In [2]:
# load the google api key from the environment variable
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.environ.get("GOOGLE_API_KEY")
if not google_api_key:
    print(
        "GOOGLE_API_KEY is not set. Create one in Google AI Studio "
        "(https://aistudio.google.com/) and add it to your env or .env file."
    )
    raise SystemExit("Set GOOGLE_API_KEY and rerun this cell.")

print("✅ GOOGLE_API_KEY loaded.")


✅ GOOGLE_API_KEY loaded.


### Import ADK components

In [3]:
from google import genai
from google.genai import types

client = genai.Client(api_key=google_api_key)

try:
    _ = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents="ping",
        config=types.GenerateContentConfig(max_output_tokens=1),
    )
    print("✅ API key validated with Gemini.")
except Exception as err:
    raise SystemExit(f"API key validation failed: {err}")


✅ API key validated with Gemini.


### Import ADK components

In [4]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## Part 2: Building an Agent with ADK  

### What is an Agent?
 
When interacting with a Large Language Model (LLM) like Gemini or GPT, the standard workflow is:

```
Prompt  -->  LLM  -->  Response (Text output)
```

 **Agents** extend this paradigm. Rather than just generating static text, agents have the ability to reason step-by-step, call external tools, observe results, and synthesize a final answer. In essence, an agent can *think*, *act*, *observe the world*, and *improve its answers* based on those observations.

A typical agent workflow:
 
```
Prompt  -->  Agent  -->  [Thought -> Action -> Observation]*  -->  Final Answer
```
- **Thought:** The agent reasons about the next step.
- **Action:** Calls a tool or takes an external action (such as searching the web).
- **Observation:** Receives results/outputs from that action and integrates them.
- The cycle can repeat (*), allowing iterative improvement.

With the [Google ADK](https://github.com/google/adk), you can easily define such agents and equip them with "tools" like `google_search`, APIs, or custom functions.

In this notebook, you'll build an agent that leverages **Google Search** as a tool, allowing it to answer questions with up-to-date, real-world information. 

### Defining an Agent 
To create an agent, we specify a few core properties:

- **name** and **description**: A human-readable name and description to identify the agent and describe its purpose.
- **model**: The underlying LLM that powers the agent's reasoning. Here, we'll use `"gemini-2.5-flash-lite"` for efficiency and up-to-date language capabilities.
- **instruction**: The agent's guiding prompt. This gives the agent its goals and instructs it on how to assist users.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) the agent can use to take actions beyond just language. For our example, we'll start with the `google_search` tool, enabling the agent to access real-time, web-based information. 
Setting these properties lets us customize our agent's abilities, behavior, and access to external resources.


In [5]:
root_agent = Agent(
    name="helpful_assistant",
    model="gemini-2.5-flash-lite",
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [6]:
tool_names = [getattr(tool, "name", getattr(tool, "__name__", str(tool))) for tool in root_agent.tools]
print("Tools configured:", tool_names)

Tools configured: ['google_search']


### Run Your Agent
Let's put the agent into action! To interact with the agent, we'll use a [`Runner`](https://google.github.io/adk-docs/runtime/), the core orchestrator in ADK. The Runner is responsible for handling message flow, managing agent reasoning steps, and returning responses.

Here, we'll create an instance of `InMemoryRunner` and associate it with our `root_agent`:

In [9]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [10]:
response = await runner.run_debug("What is the capital of France? and what is the weather there?")


 ### Created new session: debug_session_id

User > What is the capital of France? and what is the weather there?
helpful_assistant > The capital of France is Paris.

The weather in Paris is expected to be a mix of sun and clouds with a high of 11°C and a low of 9°C today. There is a 30% chance of rain. Tomorrow, Thursday, the forecast is for sunny intervals with a high of 15°C and a low of 11°C. Friday is predicted to have light rain with a high of 12°C and a low of 8°C.


### Quick run and inspect steps
Run a fresh query and inspect any tool calls from the runner response.

In [11]:
fresh_response = await runner.run_debug(
    "Give me today's top technology headline. Include source."
)

answer = getattr(fresh_response, "final_response", None) or getattr(
    fresh_response, "final", None
) or fresh_response
print("Agent answer:\n", answer)

steps = getattr(fresh_response, "steps", None) or getattr(
    fresh_response, "history", None
)
if steps:
    for idx, step in enumerate(steps, start=1):
        print(f"\nStep {idx}:")
        print(step)
else:
    print("No step details exposed.")


 ### Continue session: debug_session_id

User > Give me today's top technology headline. Include source.
helpful_assistant > Here is a top technology headline for today, December 17, 2025:

"Meta brings Instagram Reels to TVs, taking a first step toward competing with YouTube." This development means Meta is launching a dedicated app for smart TVs, focusing on Reels content. The company states this is a response to user feedback suggesting that watching Reels together is more enjoyable.
Agent answer:
 [Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      text="""Here is a top technology headline for today, December 17, 2025:

"Meta brings Instagram Reels to TVs, taking a first step toward competing with YouTube." This development means Meta is launching a dedicated app for smart TVs, focusing on Reels content. The company states this is a response to user feedback suggesting that watching Reels together is more enjoyable."""
    ),
  ],
  role='model